In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sns

#import pylab
%matplotlib inline
%pylab inline

In [ ]:
# Turning off warnings for neatness
import warnings
warnings.filterwarnings('ignore')

## Introduction

In this notebook, we demonstrate a few approaches to testing the differences in results, based on different treatments.

This is done on an example of data collected by Yandex. The data represents user activity on "control" and "experimental" versions of the page.

Methods used:
- Bootstrapping
- Basic T-Test
- Mann-Whitney test

Notebook demonstrates a few important points:
 - T-Test should only be used after its assumputions are satisfied.
 - There are methods which allow us to perform testing for data which is not normally distributed.
 - The fact that there is a significant difference between the sample is **not** enough to make a correct decision, and researched should take care to perform required additional analysis.

In [ ]:
# Reading data
Raw=pd.read_csv("ab_browser_test.csv")

## Data Description & Basic Expolration

Data comes from an experiment run by Yandex (Russia's most used search engine). The goal of the experiment was to explore if changes to a website affect users actions on the website.

Dataset consists of the following variables:
- **userID**: user's unique ID
- **browser**: browser used by the user
- **slot**: control or experimental group
- **n_clicks**: number of clicks user made in n_queries
- **n_queries**: number of queries a user has run
- **n_nonclk_queries**: number of queries where the user did not click any results provided

Our goal is to identify if changes in website positively affected the users.

**We will concentrate our analysis on n_clicks**, however other variables are certainly to be analyzed at a later stage as well.

Additionally, we will try identifiying if there are any additional insights in the data.

In [ ]:
print("Rows in file = %.0f, Columns in file=%.0f" % Raw.shape)
Raw.head()

In [ ]:
# Let's compare the mean number of clicks between the two groups
Total_Clicks_Per_Group=Raw.groupby("slot").n_clicks.aggregate(np.mean)

print("Mean number of clicks in exp group is %.3f%% higher than in control" %
      ((Total_Clicks_Per_Group["exp"]*1.0/Total_Clicks_Per_Group["control"]-1)*100))

It seems that in experimental group, number of clicks was indeed higher. The difference is not very big - just a couple of percent, but in a digital world it is a lot.

If we prove that this difference is significant later, we can state that proposed change is to be recommended for implementation.

In [ ]:
sns.kdeplot(Raw.loc[Raw.slot=="control","n_clicks"],label="Control Group")
sns.kdeplot(Raw.loc[Raw.slot!="control","n_clicks"],label="Experimental Group")
plt.legend();

As seen from the chart, number of clicks distribution is far from normal, with a very heavy right tail. It looks more like Poisson, which is to be expected, given the count nature of the data.

## Bootstrapping

Bootstrapping will serve two goals:
1. Will allow us to test for diffence between the distributions, without paying much attention to its normality
2. Allow us to test t-test requirements, to see if we can actually use it


### T-test requirements

T-test has two basic requirements:
1. Sample mean is distributed normally, or approximately normally
2. Sample variance estimate is distributed as chi-sq with n-1 degrees of freedom

We will test both of those, to see if we could use t-test on data

In [ ]:
## Define bootstrapping function, and function to use its results
def get_bootstrap_samples(data, n_samples):
    data=np.array(data)
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals_from_bootstrap(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [ ]:
# Split the data
Exp_Group=Raw[Raw.slot=="exp"]
Control_Group=Raw[Raw.slot=="control"]

# Create bootstraps
Exp_click_bootstraps=get_bootstrap_samples(Exp_Group.n_clicks,500)
Control_click_bootstraps=get_bootstrap_samples(Control_Group.n_clicks,500)

# Calculate means by group from bootstrapped samples
Exp_means=map(np.mean,Exp_click_bootstraps)
Control_means=map(np.mean,Control_click_bootstraps)

# Calculate bootstrapped differences in means between two groups
delta_means = list(map(lambda x: x[1] - x[0], zip(Exp_means, Control_means)))

In [ ]:
# Output bootstrapped confidence interval for means
print("Confidence interval for mean difference : [%.3f, %.3f]" %
      tuple(stat_intervals_from_bootstrap(list(delta_means),0.05)))

### Bootstrap mean comparison result
As we see, there is no 0 in bootstraped interval for mean difference. As such, we can say already that there is significant difference between the two groups.

### T-Test requirements
To check T-test requirements, we will use control group, since distributions in the two groups are rather similar. We will get distribution of sample mean and sample variance estimates, and see if they are distributed according to T-test requirements

In [ ]:
## Calculate distribution of means and variances
Control_means=list(map(np.mean,Control_click_bootstraps))
Control_variance=list(map(np.var,Control_click_bootstraps))


## Create QQ plots for both distributions, comparing
# means with normal
# variances with chisq
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,6))


Coords,Fit=stats.probplot(Control_means, dist="norm", plot=axes[0],fit=True)
axes[0].set_title('Probability Plot for Means R^2 = %.5f' % (Fit[2]**2))


Coords,Fit=stats.probplot(Control_variance, dist="chi2", 
                     sparams=(len(Control_Group.n_clicks)-1), plot=axes[1],fit=True)
axes[1].set_title('Probability Plot for Variances R^2 = %.5f' % (Fit[2]**2))

plt.show()

### T-Test requirements results
Both based on visual analysis of QQ plots, and R^2 obtained, we can conclude that T-test requirements are satisfied - the distributions are quite close to the ones required.

## T-Test

Since T-Test requirements are satisfied, we can use this for testing

In [ ]:
print(stats.ttest_ind(Exp_Group.n_clicks,Control_Group.n_clicks,equal_var=False))

Clearly, T-Test also shows us that difference between in number of click is signficiant between the samples.

## Mann-Whitney test
Now, let's pretend that T-Test requirements are not satisfied, and the amount of data is a bit too big to run bootstrapping.
A good option in that case would be to use Mann-Whitney test. Its requirements are not as strict as T-test's, and it may be more efficient on a wide range of distributions.

In [ ]:
stats.mannwhitneyu(Exp_Group.n_clicks,Control_Group.n_clicks,alternative="two-sided")

Yet another proof that difference between the two samples is significant (not that we needed another one, but its a learning excersize after all).

## Depper dive
A poorly-minded data scientist would stop at this point, and say that changes are positive and should be implemented straight away.

However, we are better than this. We have another variable in the dataset - browser. Perhaps, it is a good idea to check if the differences are consistent by browser.

In [ ]:
for browser in Raw.browser.unique():
    ThisPiece=Raw[Raw.browser==browser]
    
    Total_Clicks_Per_Group=ThisPiece.groupby("slot").n_clicks.aggregate(np.mean)

    print("Difference for %s %.3f%% higher than in control" %
          (browser,((Total_Clicks_Per_Group["exp"]*1.0/Total_Clicks_Per_Group["control"]-1)*100)))

Aha! We can see that difference is only really noticeably positive for Browser #14. We have already stated though, that smaller differences may be quite important as well - let's see if they are significant.

In [ ]:
for browser in Raw.browser.unique():
    ThisPiece=Raw[Raw.browser==browser]
    
    ThisPieceExp_Group=ThisPiece[ThisPiece.slot=="exp"]
    ThisPieceControl_Group=ThisPiece[ThisPiece.slot=="control"]
    
    print("P-value for Difference for %s is %.3f##" % 
        (browser,
         stats.mannwhitneyu(ThisPieceExp_Group.n_clicks,ThisPieceControl_Group.n_clicks,alternative="two-sided")[1]))


## Results

### Method demonstration 
We have demonstrated usage of three different methods for sig testing AB-experiment results. In this particular case, all of the tests we used were appropriate for the data, and yielded similar results.

It is important to notice though, that:
 - Bootstrapping did take some significant amount of time to run - it might be unwise to run in on sample of such size
 - We have pointed out that before using T-Test, we need to check if its requirements are satisfied. However, we do not always have an option to run bootstrapping. In this case, it would've been better to check for data normality, and since our data is not normally distributed Mann-Whitney would've been considered a better option.
 - Thoughtless acceptance of total-level result without subgroup analysis would've potentially had a very negative result - we would've implemented a change which might negatively affect users of one of the browsers

### Practical
As expected, difference is only significant for Browser 14. Hence, we can only recommed for the change to be implemented for users on this particular browser.

However, from a useability point of view, it might be of course confusing for users to have different experiences on different browsers. If that's the case, we would rather implement the change for all of them. But we can see that for Browser #22 there is a negative effect, which is worrysome. If we would rather implement changes for all browsers, it is recommended to collect more sample, to check if for Browser #22 users we do not make the experience worse.